In [1]:
from setup import get_openai_model, enable_tracing
from chains.content_chain  import build_content_chain
from chains.message_chain  import build_message_chain

MODEL = get_openai_model()
enable_tracing("chat_image")

content_chain = build_content_chain(MODEL)
message_chain = build_message_chain(MODEL)

OPENAI ready
Enable tracing at chat_image


In [2]:
prompt_infos = [
    {
        "name": "message",
        "description": "Good for content is about human text messages",
        "chain": message_chain,
    },
    {
        "name": "content",
        "description": "Good for content is about a book, a review, news,...",
        "chain": content_chain,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    chain = p_info['chain']
    destination_chains[name] = chain
default_chain = destination_chains['message']

In [3]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.prompts import PromptTemplate
from langchain.chains.router import MultiPromptChain

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(MODEL, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [9]:
ans = chain({"input":"You're so ugly, I hate you so much."})
ans



> Entering new  chain...
None: {'input': "You're so ugly, I hate you so much."}

> Entering new  chain...
Prompt after formatting:
You are good at detecting harmful, content and also encouraging good and useful one. All labels that you know are Negative and Positive. Given a past content, your job is to answer as best as possible.
The past message:
You're so ugly, I hate you so much.

Instructions:
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"sentiment": string  // a sentiment label based on the past content. It should be either Negative or Positive
	"ideas": string  // a js array of response options to the given content. These options must help the user to feel better.
	"chain": string  // always return 'message'
}
```
The question: What do you think about the message.

> Finished chain.

> Finished chain.


{'input': "You're so ugly, I hate you so much.",
 'text': '\n\n```json\n{\n\t"sentiment": "Negative",\n\t"ideas": "[\\"It\'s ok to feel this way, but it\'s important to remember that everyone has their own unique beauty.\\", \\"It\'s ok to feel this way, but it\'s important to remember that everyone has something special about them.\\", \\"It\'s ok to feel this way, but it\'s important to remember that everyone has something to offer.\\"]",\n\t"chain": "message"\n}\n```'}

In [8]:
dir(chain)

['Config',
 '__abstractmethods__',
 '__annotations__',
 '__call__',
 '__class__',
 '__class_vars__',
 '__config__',
 '__custom_root_type__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__exclude_fields__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_validators__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__include_fields__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__json_encoder__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_root_validators__',
 '__pre_root_validators__',
 '__pretty__',
 '__private_attributes__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__repr_args__',
 '__repr_name__',
 '__repr_str__',
 '__rich_repr__',
 '__schema_cache__',
 '__setattr__',
 '__setstate__',
 '__signature__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__try_update_forward_refs__',
 '__validators__',
 '_abc_impl',
 '_acall',
 '_calculate_keys',
 '_call',
 '_chain_type',
 '_